# 使用tensorboard绘制神经网络的网络结构

In [2]:
# 手写数字识别案例(线性分类器)
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# 载入数据集。第一个参数是路径名，第二个是采用的编码
mnist = input_data.read_data_sets("/Users/yuejinxiong/MNIST_data",one_hot=True)
# 每个批次的大小(每次放入100张图片去训练)
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

# 定义一个命名空间 起名为input，用于可视化网络结构
# 在这个名为input的命名空间下有两个变量，分别名为x-input、y-input
with tf.name_scope("input"):
    # 定义两个placeholder
    x = tf.placeholder(tf.float32, [None,784], name="x-input")
    y = tf.placeholder(tf.float32, [None,10], name="y-input")

# 第二个命名空间
with tf.name_scope("layer"):
    # 创建一个简单的神经网络--只有输入和输出层，没有隐藏层（改进后增加隐藏层）
    with tf.name_scope("wights"):
        W = tf.Variable(tf.zeros([784, 10]),name="W")
    with tf.name_scope("biases"):
        b = tf.Variable(tf.zeros([10]),name="b")
    with tf.name_scope("wx_plus_b"):
        wx_plus_b = tf.matmul(x, W)+b
    with tf.name_scope("softmax"):
        prediction = tf.nn.softmax(wx_plus_b)

# 增加了隐藏层之后,参数变多了,更加难以收敛,如果想得到好的结果,必须要迭代很多次
# 隐藏层的神经元个数最好是2^n个

# 输入层与隐藏层之间的权重和偏置的初始化 隐藏层有5个神经元
# Weight_L1 = tf.Variable(tf.random_normal([784, 512]))
# biases_L1 = tf.Variable(tf.zeros([1, 512]))
# Wx_plus_b_L1 = tf.matmul(x, Weight_L1) + biases_L1
# L1 = tf.nn.tanh(Wx_plus_b_L1)

# 隐藏层和输出层之间权重和偏置的初始化
# Weight_L2 = tf.Variable(tf.random_normal([512, 10]))
# biases_L2 = tf.Variable(tf.zeros([1, 10]))
# Wx_plus_b_L2 = tf.matmul(L1, Weight_L2) + biases_L2
# prediction = tf.nn.softmax(Wx_plus_b_L2)

# 二次代价函数
# loss = tf.reduce_mean(tf.square(y-prediction))
# 现在改为对数似然函数来优化
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))
# 梯度下降
with tf.name_scope("train"):
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()

# 这里求预测的正确数、结果存在一个布尔型的列表中
# equal(num1, num2)比较两个参数的大小是否一样,返回值为True或False
# arg_max() 求y(一维张量)这个结果最大的值的索引位置(如果标签值和预测值最大的值在同一个位置，则说明该条正确) 1表示按行查找
with tf.name_scope("accuracy"):
    with tf.name_scope("correct_prediction"):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
    with tf.name_scope("accuracy"):
    # 求准确率, cast()将布尔类型的列表中的元素转成32位的浮点类型,然后再求一个平均值(true=1.0 false=0)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(init)
    # 第一个参数是路径,存放绘制的图的结构
    writer = tf.summary.FileWriter('structure_graph/', sess.graph)
    # 所有数据迭代21次
    for epoch in range(1):
        # 一次迭代中迭代计算好的数量个批次
        for batch in range(n_batch):
            # 获取100个批次保存在里面,数据保存在batch_xs中，标签保存在batchys中
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x:batch_xs, y:batch_ys})
        # 训练一个周期后,可以查看其准确率的变化
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print("第"+str(epoch+1)+"次迭代的准确率: "+str(acc))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/yuejinxiong/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/yuejinxiong/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /Users/yuejinxiong/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/yuejinxiong/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
第1次迭代的准确率: 0.8247


# 上面代码将图生成好了之后
 - 在控制台输入下面代码打开浏览器即可
 `tensorboard --logdir=/Users/yuejinxionTensorflow_learning/structure_graph/`

# 查看网络运行过程的中数据

In [1]:
# 手写数字识别案例(线性分类器)
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# 载入数据集。第一个参数是路径名，第二个是采用的编码
mnist = input_data.read_data_sets("/Users/yuejinxiong/MNIST_data",one_hot=True)
# 每个批次的大小(每次放入100张图片去训练)
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

# 参数摘要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)  # 求平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)  #求标准差
        tf.summary.scalar('max', tf.reduce_max(var))  #最大值
        tf.summary.scalar('min', tf.reduce_min(var))  #最小值
        tf.summary.histogram('histogram', var)  #直方图

# 定义一个命名空间 起名为input，用于可视化网络结构
# 在这个名为input的命名空间下有两个变量，分别名为x-input、y-input
with tf.name_scope("input"):
    # 定义两个placeholder
    x = tf.placeholder(tf.float32, [None,784], name="x-input")
    y = tf.placeholder(tf.float32, [None,10], name="y-input")

# 第二个命名空间
with tf.name_scope("layer"):
    # 创建一个简单的神经网络--只有输入和输出层，没有隐藏层（改进后增加隐藏层）
    with tf.name_scope("wights"):
        W = tf.Variable(tf.zeros([784, 10]),name="W")
        # 如果想分析下这个权值，可以传入到函数中
        variable_summaries(W)
    with tf.name_scope("biases"):
        b = tf.Variable(tf.zeros([10]),name="b")
        variable_summaries(b)
    with tf.name_scope("wx_plus_b"):
        wx_plus_b = tf.matmul(x, W)+b
    with tf.name_scope("softmax"):
        prediction = tf.nn.softmax(wx_plus_b)

# 增加了隐藏层之后,参数变多了,更加难以收敛,如果想得到好的结果,必须要迭代很多次
# 隐藏层的神经元个数最好是2^n个

# 输入层与隐藏层之间的权重和偏置的初始化 隐藏层有5个神经元
# Weight_L1 = tf.Variable(tf.random_normal([784, 512]))
# biases_L1 = tf.Variable(tf.zeros([1, 512]))
# Wx_plus_b_L1 = tf.matmul(x, Weight_L1) + biases_L1
# L1 = tf.nn.tanh(Wx_plus_b_L1)

# 隐藏层和输出层之间权重和偏置的初始化
# Weight_L2 = tf.Variable(tf.random_normal([512, 10]))
# biases_L2 = tf.Variable(tf.zeros([1, 10]))
# Wx_plus_b_L2 = tf.matmul(L1, Weight_L2) + biases_L2
# prediction = tf.nn.softmax(Wx_plus_b_L2)

# 二次代价函数
# loss = tf.reduce_mean(tf.square(y-prediction))
# 现在改为对数似然函数来优化
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))
    tf.summary.scalar('loss', loss)
# 梯度下降
with tf.name_scope("train"):
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()

# 这里求预测的正确数、结果存在一个布尔型的列表中
# equal(num1, num2)比较两个参数的大小是否一样,返回值为True或False
# arg_max() 求y(一维张量)这个结果最大的值的索引位置(如果标签值和预测值最大的值在同一个位置，则说明该条正确) 1表示按行查找
with tf.name_scope("accuracy"):
    with tf.name_scope("correct_prediction"):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
    with tf.name_scope("accuracy"):
    # 求准确率, cast()将布尔类型的列表中的元素转成32位的浮点类型,然后再求一个平均值(true=1.0 false=0)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('accuracy', accuracy)

# 合并所有的summary
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    # 第一个参数是路径,存放绘制的图的结构
    writer = tf.summary.FileWriter('structure_graph/', sess.graph)
    # 所有数据迭代21次
    for epoch in range(51):
        # 一次迭代中迭代计算好的数量个批次
        for batch in range(n_batch):
            # 获取100个批次保存在里面,数据保存在batch_xs中，标签保存在batchys中
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            summary,_ = sess.run([merged,train_step], feed_dict={x:batch_xs, y:batch_ys})
        writer.add_summary(summary, epoch)
        # 训练一个周期后,可以查看其准确率的变化
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print("第"+str(epoch+1)+"次迭代的准确率: "+str(acc))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/yuejinxiong/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/yuejinxiong/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /Users/yuejinxiong/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/yuejinxiong/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
第1次迭代的准确率: 0.8236
第2次迭代的准确率: 0.8924
第3次迭代的准确率: 0.899
第4次迭代的准确率: 0.9061
第5次迭代的准确率: 0.9082
第6次迭代的准确率: 0.9097
第7次迭代的准确率: 0.9117
第8次迭代的准确率: 0.9132
第9次迭代的准确率: 0.9157
第10次迭代的准确率: 0.9161
第11次迭代的准确率: 0.9165
第12次迭代的准确率: 0.9184
第13次迭代的